In [154]:
import math

import osmium, cairo

def merkator(lon, lat):
    lon *= math.pi / 180
    lat *= math.pi / 180
    r = 6371
    return (
        r * lon,
        r * math.log(math.tan(math.pi / 4 + lat / 2)),
    )

def img_coords_factory(clon, clat, km_width, px_width, ratio):
    cx, cy = merkator(clon, clat)
    scale = px_width / km_width * math.cos(clat / 180 * math.pi)
    px_height = px_width / ratio
    def fun(lon, lat):
        x, y = merkator(lon, lat)
        return (
            (x - cx) * scale + px_width / 2,
            px_height / 2 - (y - cy) * scale
        )
    return fun

def ensure_direction(pts, clockwise):
    s = 0
    for i in range(len(pts) - 1):
        x1, y1 = pts[i]
        x2, y2 = pts[i + 1]
        s += (x2 - x1) * (y2 + y1)
    if (s < 0) != clockwise:
        return list(reversed(pts))
    return pts

def pts(locs):
    return [
        img_coords(loc.lon, loc.lat)
        for loc in locs
    ]

def get_id(fig):
    return fig[{0: 3, 1: 2, 2: 1, 3: 2}[fig[0]]]

def to_way(fig):
    if fig[0] == 1:
        return fig
    elif fig[0] == 3:
        if len(fig[1]) != 1:
            print('fig %d is not a way' % get_id(fig))
        return (1, fig[1][0], get_id(fig))
    else:
        print('fig %d type = %d' % (get_id(fig), fig[0]))

class MapDrawer(osmium.SimpleHandler):

    def __init__(self):
        osmium.SimpleHandler.__init__(self)
        self.op = {
            'spec_landuse': [],
            'buildings': [],
            'barriers': [],
            'priv_gates': [],
            'priv_roads': [],
            'footways': [],
            'tertiary': [],
            'secondary': [],
            'grass': [],
            'public': [],
            'entrances': [],
        }
    
    def check_obj(self, fig, tags):
        if tags.get('building'):
            self.op['buildings'].append((fig, tags.get('building:levels')))
            return
        if tags.get('landuse') in ['construction', 'industrial', 'military']:
            self.op['spec_landuse'].append(fig)
            return
        if tags.get('natural') in ['water']:
            self.op['barriers'].append(fig)
            return            
        if tags.get('entrance'):
            self.op['entrances'].append(fig)
            return
        if tags.get('access') in ['private', 'no'] and tags.get('foot') not in ['yes', 'designated']:
            if fig[0] == 0:
                self.op['priv_gates'].append(to_way(fig) if fig[0] == 3 else fig)                
            else:
                self.op['priv_roads'].append(to_way(fig) if fig[0] == 3 else fig)
            return
        if tags.get('landuse') == 'grass' or tags.get('leisure') in ['park', 'dog_park'] or tags.get('natural') in ['wood']:
            self.op['grass'].append(fig)
        if tags.get('barrier') and tags.get('barrier') not in ['kerb']:
            if fig[0] == 0:
                self.op['footways'].append(fig)
            else:
                self.op['barriers'].append(to_way(fig) if fig[0] == 3 else fig)
        if tags.get('highway'):
            if tags.get('highway') in ['motorway', 'motorway_link', 'trunk', 'trunk_link', 'primary', 'primary_link', 'secondary', 'secondary_link']:
                self.op['secondary'].append(to_way(fig) if fig[0] == 3 else fig)
            elif tags.get('highway') in ['tertiary', 'tertiary_link', 'unclassified', 'residential']:
                self.op['tertiary'].append(to_way(fig) if fig[0] == 3 else fig)
            else:
                self.op['footways'].append(to_way(fig) if fig[0] == 3 else fig)
            return
        # leisure: park, dog_park
        # natural: wood

    def node(self, n):
        loc = n.location
        x, y = img_coords(loc.lon, loc.lat)
        self.check_obj((0, x, y, n.id), n.tags)

    def way(self, w):
        if w.ends_have_same_id():
            return
        pts = []
        for node in w.nodes:
            pts.append(img_coords(node.lon, node.lat))
        self.check_obj((1, pts, w.id), w.tags)

    def relation(self, r):
        if not r.tags.get('type').startswith('multipolygon'):
            self.check_obj((2, r.id), r.tags)
    
    def area(self, a):
        rings = []
        has_inner = False
        for outer_ring in a.outer_rings():
            rings.append(ensure_direction(
                pts(outer_ring), True
            ))
            for inner_ring in a.inner_rings(outer_ring):
                has_inner = True
                rings.append(ensure_direction(
                    pts(inner_ring), False
                ))
        self.check_obj((3, rings, a.id), a.tags)

def draw(ctx, fig, rad, linewidth_way, fill, stroke_col=None):
    if fig[0] == 0:
        ctx.set_source_rgba(*fill)
        ctx.arc(fig[1], fig[2], rad, 0, 2 * math.pi)
        ctx.fill()
    elif fig[0] == 1:
        ctx.set_line_width(linewidth_way)
        pts = fig[1]
        ctx.move_to(pts[0][0], pts[0][1])
        for pt in pts:
            ctx.line_to(pt[0], pt[1])
        ctx.set_source_rgba(*fill)
        ctx.stroke()
    elif fig[0] == 2:
        print(fig[1])
    elif fig[0] == 3:
        rings = fig[1]
        for ring in rings:
            ctx.move_to(ring[0][0], ring[0][1])
            for pt in ring:
                ctx.line_to(pt[0], pt[1])
        ctx.set_source_rgba(*fill)
        if stroke_col is not None:
            ctx.fill_preserve()
            ctx.set_line_width(1)
            ctx.set_source_rgba(*stroke_col)
            ctx.stroke()
        else:
            ctx.fill()

def rect(x1, y1, x2, y2):
    return [(x1, y1), (x1, y2), (x2, y2), (x2, y1), (x1, y1)]

def draw_map(op, colors, img_size):
    surface = cairo.ImageSurface(colors['format'], img_size[0], img_size[1])
    ctx = cairo.Context(surface)
    ctx.set_antialias(cairo.Antialias.NONE)
    ctx.set_fill_rule(cairo.FillRule.WINDING)
    for fig in op['spec_landuse']:
        draw(ctx, fig, 3, 2, colors['spec_landuse'])
    for fig, levels in op['buildings']:
        if type(levels) == str and levels.isdigit():
            levels = int(levels)
        else:
            levels = 0
        draw(ctx, fig, 3, 2, colors['buildings'][levels], colors['gates'])
    for fig in op['entrances']:
        draw(ctx, fig, 2, 2, colors['entrances'])
    for fig in op['barriers']:
        draw(ctx, fig, 3, 3, colors['barriers'])
    ctx.save()
    ctx.set_operator(cairo.Operator.ATOP)
    for fig in op['priv_roads']:
        draw(ctx, fig, 3, 2, colors['barriers'])
    ctx.set_operator(cairo.Operator.DEST_OVER)
    for fig in op['priv_roads']:
        draw(ctx, fig, 3, 2, colors['footways'])
    ctx.restore()
    ctx.save()
    ctx.set_operator(cairo.Operator.DEST_OVER)
    for fig in op['secondary']:
        draw(ctx, fig, 3, 2, colors['gates'])
    for fig in op['tertiary']:
        draw(ctx, fig, 3, 1, colors['gates'])
    for fig in op['secondary']:
        pass
        #draw(fig, 3, 6, colors['footways'], True)
    for fig in op['tertiary']:
        draw(ctx, fig, 3, 6, colors['footways'])
    ctx.restore()
    for fig in op['footways']:
        draw(ctx, fig, 2, 2, colors['footways'])
    for fig in op['priv_gates']:
        draw(ctx, fig, 2, 2, colors['gates'])
    ctx.save()
    ctx.set_operator(cairo.Operator.DEST_OVER)
    for fig in op['grass']:
        draw(ctx, fig, 3, 2, colors['grass'])
    ctx.restore()
    m = (250, 225, 400, 300)
    draw(ctx, (1, rect(m[0], m[1], imgw - m[2], imgh - m[3]), -1), 3, 2, colors['barriers'])
    ctx.save()
    ctx.set_operator(cairo.Operator.DEST_OVER)
    ctx.set_source_rgba(*colors['default'])
    ctx.rectangle(0, 0, img_size[0], img_size[1])
    ctx.fill()
    ctx.restore()
    return surface

imgw = 2700
imgh = 2100
img_coords = img_coords_factory(37.5376, 55.8003, 4.5, imgw, imgw / imgh)

h = MapDrawer()
h.apply_file('./aeroport.osm', locations=True)

colors_rgb = {
    'default': (1, 1, 1, 1),
    'spec_landuse': (0, 0, 1, 1),
    'buildings': [((1 + levels) / 128 + 1/2,) * 3 + (1,) for levels in range(128)],
    'entrances': (1, 0.5, 0, 1),
    'barriers': (0, 0, 0, 1),
    'footways': (0, 1, 0, 1),
    'gates': (1, 0, 0, 1),
    'grass': (0, 0.5, 0, 1),
    'format': cairo.FORMAT_ARGB32,
}

colors_gray = {
    'barriers': 0,
    'gates': 20,
    'entrances': 40,
    'spec_landuse': 60,
    'grass': 80,
    'default': 100,
    'footways': 120,
    'buildings': [128 + levels for levels in range(128)],
    'format': cairo.FORMAT_ARGB32,
}

for cname in colors_gray:
    if cname == 'format':
        continue
    g = colors_gray[cname]
    if cname == 'buildings':
        colors_gray[cname] = [(l / 256, l / 256, l / 256, 1) for l in g]
    else:
        colors_gray[cname] = (g / 256, g / 256, g / 256, 1)

surface = draw_map(h.op, colors_rgb, [imgw, imgh])
surface.write_to_png('map.png')
surface = draw_map(h.op, colors_gray, [imgw, imgh])
with open('map.pgm', 'wb') as f:
    f.write(surface.get_data().tobytes()[::4])

In [3]:
1021 * 2700 + 1107

2757807

In [157]:
5670000

5670000